In [ ]:
import numpy as np
import os
import pickle
import torch
from iopath.common.file_io import g_pathmgr

import slowfast.utils.checkpoint as cu
import slowfast.utils.distributed as du
import slowfast.utils.logging as logging
import slowfast.utils.misc as misc
import slowfast.visualization.tensorboard_vis as tb
from slowfast.datasets import loader
from slowfast.models import build_model
from slowfast.utils.meters import AVAMeter, TestMeter


In [ ]:
logger = logging.get_logger(__name__)

In [ ]:
@torch.no_grad()
def perform_test(test_loader, model, test_meter, cfg, writer=None):
    pass